In [1]:
!pip install torch torchvision
!pip install git+https://github.com/facebookresearch/detectron2.git

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.5.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.20.0-cp312-cp312-win_amd64.whl.metadata (6.2 kB)
Using cached torch-2.5.0-cp312-cp312-win_amd64.whl (203.1 MB)
Using cached torchvision-0.20.0-cp312-cp312-win_amd64.whl (1.6 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.0+cu118 requires torch==2.5.0+cu118, but you have torch 2.5.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\alaka\appdata\local\temp\pip-req-build-23h_u407
  Resolved https://github.com/facebookresearch/detectron2.git to commit 8d85329aed8506ea3672e3e208971345973ea761
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pycocotools-2.0.8-cp312-cp312-win_amd64.whl.metadata (1.1 kB)
  Using cached yacs-0.1.8-py3-none-any.whl.metadata (639 bytes)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached iopath-0.1.9-py3-none-any.whl.metadata (370 bytes)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached black-24.10.0-cp312-cp312-win_amd64.whl.metadata (79 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any

  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\alaka\AppData\Local\Temp\pip-req-build-23h_u407'
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [368 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\detectron2
      copying detectron2\__init__.py -> build\lib.win-amd64-cpython-312\detectron2
      creating build\lib.win-amd64-cpython-312\tools
      copying tools\analyze_model.py -> build\lib.win-amd64-cpython-312\tools
      copying tools\benchmark.py -> build\lib.win-amd64-cpython-312\tools
      copying tools\convert-torchvision-to-d2.py -> build\lib.win-amd64-cpython-312\tools
      copying tools\lazyconfig_train_net.py -> build\lib.win-amd64-cpython-312\tools
      copying tools\lightning_tra

In [ ]:
!git clone https://github.com/facebookresearch/meshrcnn.git
%cd meshrcnn
!pip install -e .


In [2]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Load the image
image_path = "/content/hi.jpg"
image = cv2.imread(image_path)

# Configure Mask R-CNN
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

# Run Mask R-CNN prediction
outputs = predictor(image)

# Visualize Mask R-CNN Results
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis("off")
plt.show()

# Extract Mask R-CNN results for Mesh R-CNN
instances = outputs["instances"]
masks = instances.pred_masks.cpu().numpy()
boxes = instances.pred_boxes.tensor.cpu().numpy()
classes = instances.pred_classes.cpu().numpy()

# Initialize Mesh R-CNN
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/meshrcnn_R50_FPN.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/meshrcnn_R50_FPN.yaml")
mesh_predictor = DefaultPredictor(cfg)

# Iterate over each detected object to generate 3D meshes
for i, (mask, box, cls) in enumerate(zip(masks, boxes, classes)):
    print(f"Processing object {i+1} (Class: {cls})")
    
    # Mask and bounding box preprocessing for Mesh R-CNN
    # Create a masked image to pass to Mesh R-CNN
    masked_image = image.copy()
    masked_image[~mask] = 0  # Mask out background pixels

    # Run Mesh R-CNN prediction on the masked image
    mesh_output = mesh_predictor(masked_image)
    
    # Process mesh_output as required (e.g., visualize or save as 3D model)
    # (In this example, we'll visualize using Mesh R-CNN's output format)
    v = Visualizer(masked_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(mesh_output["instances"].to("cpu"))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.axis("off")
    plt.show()

    # Optionally save the mesh to a file (requires further mesh processing)
    # Example: save_mesh_to_file(mesh_output, f"object_{i+1}_mesh.obj")

print("Completed Mask R-CNN and Mesh R-CNN processing.")

ModuleNotFoundError: No module named 'cv2'